A two-stage model (retreiver and reranker) would be optimal.  
This notebook confirms the improvement in scores by the only ensemble of diverse retreivers,
* all-mpnet-base-v2
* all-MiniLM-L6-v2

And the reranker ensemble is also supposed to be promising.
Stay tuned...

Reference  
* Unsupervised Train Set  
https://www.kaggle.com/code/ragnar123/lecr-unsupervised-train-set-public  
* Train part  
https://www.kaggle.com/code/ragnar123/lecr-xlm-roberta-base-baseline  
* Inference part  
https://www.kaggle.com/code/ragnar123/lecr-inference-p  

Thanks.

In [1]:
# =========================================================================================
# Libraries
# =========================================================================================
import os
import gc
import time
import math
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding
import cupy as cp
from cuml.metrics import pairwise_distances
from cuml.neighbors import NearestNeighbors
%env TOKENIZERS_PARALLELISM=false
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CFG1:
    uns_model = "/kaggle/input/20230123/sentence-transformers-all-mpnet-base-v2"
    sup_model = "/kaggle/input/20230123/xlm-roberta-base"
    uns_tokenizer = AutoTokenizer.from_pretrained(uns_model + '/tokenizer')
    sup_tokenizer = AutoTokenizer.from_pretrained(sup_model + '/tokenizer')
    
class CFG2:
    uns_model = "/kaggle/input/20230123/sentence-transformers-all-MiniLM-L6-v2"
    sup_model = "/kaggle/input/20230123/xlm-roberta-base"
    uns_tokenizer = AutoTokenizer.from_pretrained(uns_model + '/tokenizer')
    sup_tokenizer = AutoTokenizer.from_pretrained(sup_model + '/tokenizer')
    
CFG_list = [CFG1, CFG2]

env: TOKENIZERS_PARALLELISM=false


In [2]:
def read_data(cfg):
    topics = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/topics.csv')
    content = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/content.csv')
    sample_submission = pd.read_csv('/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv')
    # Merge topics with sample submission to only infer test topics
    topics = topics.merge(sample_submission, how = 'inner', left_on = 'id', right_on = 'topic_id')
    # Fillna titles
    topics['title'].fillna("", inplace = True)
    content['title'].fillna("", inplace = True)
    # Sort by title length to make inference faster
    topics['length'] = topics['title'].apply(lambda x: len(x))
    content['length'] = content['title'].apply(lambda x: len(x))
    topics.sort_values('length', inplace = True)
    content.sort_values('length', inplace = True)
    # Drop cols
    topics.drop(['description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'length', 'topic_id', 'content_ids'], axis = 1, inplace = True)
    content.drop(['description', 'kind', 'language', 'text', 'copyright_holder', 'license', 'length'], axis = 1, inplace = True)
    # Reset index
    topics.reset_index(drop = True, inplace = True)
    content.reset_index(drop = True, inplace = True)
    print(' ')
    print('-' * 50)
    print(f"topics.shape: {topics.shape}")
    print(f"content.shape: {content.shape}")
    return topics, content

def prepare_uns_input(text, cfg):
    inputs = cfg.uns_tokenizer.encode_plus(
        text, 
        return_tensors = None, 
        add_special_tokens = True, 
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype = torch.long)
    return inputs

# =========================================================================================
# Unsupervised dataset
# =========================================================================================
class uns_dataset(Dataset):
    def __init__(self, df, cfg):
        self.cfg = cfg
        self.texts = df['title'].values
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, item):
        inputs = prepare_uns_input(self.texts[item], self.cfg)
        return inputs
    
# =========================================================================================
# Prepare input, tokenize
# =========================================================================================
def prepare_sup_input(text, cfg):
    inputs = cfg.sup_tokenizer.encode_plus(
        text, 
        return_tensors = None, 
        add_special_tokens = True, 
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype = torch.long)
    return inputs

# =========================================================================================
# Supervised dataset
# =========================================================================================
class sup_dataset(Dataset):
    def __init__(self, df, cfg):
        self.cfg = cfg
        self.texts = df['text'].values
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, item):
        inputs = prepare_sup_input(self.texts[item], self.cfg)
        return inputs

# =========================================================================================
# Mean pooling class
# =========================================================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

# =========================================================================================
# Unsupervised model
# =========================================================================================
class uns_model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.config = AutoConfig.from_pretrained(cfg.uns_model + '/config')
        self.model = AutoModel.from_pretrained(cfg.uns_model + '/model', config = self.config)
        self.pool = MeanPooling()
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        feature = self.pool(last_hidden_state, inputs['attention_mask'])
        return feature
    def forward(self, inputs):
        feature = self.feature(inputs)
        return feature
    
# =========================================================================================
# Get embeddings
# =========================================================================================
def get_embeddings(loader, model, device):
    model.eval()
    preds = []
    for step, inputs in enumerate(tqdm(loader)):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.to('cpu').numpy())
    preds = np.concatenate(preds)
    return preds

# =========================================================================================
# Get the amount of positive classes based on the total
# =========================================================================================
def get_pos_socre(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    int_true = np.array([len(x[0] & x[1]) / len(x[0]) for x in zip(y_true, y_pred)])
    return round(np.mean(int_true), 5)

# =========================================================================================
# Build our inference set
# =========================================================================================
def build_inference_set(tmp_topics, tmp_content, cfg):
    # Create lists for training
    topics_ids = []
    content_ids = []
    title1 = []
    title2 = []
    # Iterate over each topic
    for k in tqdm(range(len(tmp_topics))):
        row = tmp_topics.iloc[k]
        topics_id = row['id']
        topics_title = row['title']
        predictions = row['predictions'].split(' ')
        for pred in predictions:
            content_title = tmp_content.loc[pred, 'title']
            topics_ids.append(topics_id)
            content_ids.append(pred)
            title1.append(topics_title)
            title2.append(content_title)
    # Build training dataset
    tmp_test = pd.DataFrame(
        {'topics_ids': topics_ids, 
         'content_ids': content_ids, 
         'title1': title1, 
         'title2': title2
        }
    )
    # Release memory
    del topics_ids, content_ids, title1, title2
    gc.collect()
    torch.cuda.empty_cache()
    return tmp_test
    
# =========================================================================================
# Get neighbors
# =========================================================================================
def get_neighbors(tmp_topics, tmp_content, cfg):
    # Create topics dataset
    topics_dataset = uns_dataset(tmp_topics, cfg)
    # Create content dataset
    content_dataset = uns_dataset(tmp_content, cfg)
    # Create topics and content dataloaders
    topics_loader = DataLoader(
        topics_dataset, 
        batch_size = 32, 
        shuffle = False, 
        collate_fn = DataCollatorWithPadding(tokenizer = cfg.uns_tokenizer, padding = 'longest'),
        num_workers = 4, 
        pin_memory = True, 
        drop_last = False
    )
    content_loader = DataLoader(
        content_dataset, 
        batch_size = 32, 
        shuffle = False, 
        collate_fn = DataCollatorWithPadding(tokenizer = cfg.uns_tokenizer, padding = 'longest'),
        num_workers = 4, 
        pin_memory = True, 
        drop_last = False
        )
    # Create unsupervised model to extract embeddings
    model = uns_model(cfg)
    model.to(device)
    # Predict topics
    topics_preds = get_embeddings(topics_loader, model, device)
    content_preds = get_embeddings(content_loader, model, device)
    # Transfer predictions to gpu
    topics_preds_gpu = cp.array(topics_preds)
    content_preds_gpu = cp.array(content_preds)
    # Release memory
    del topics_dataset, content_dataset, topics_loader, content_loader, topics_preds, content_preds
    gc.collect()
    torch.cuda.empty_cache()
    # KNN model
    print(' ')
    print('Training KNN model...')
    neighbors_model = NearestNeighbors(n_neighbors = 1000, metric = 'cosine')
    neighbors_model.fit(content_preds_gpu)
    indices = neighbors_model.kneighbors(topics_preds_gpu, return_distance = False)
    predictions = []
    for k in range(len(indices)):
        pred = indices[k]
        p = ' '.join([tmp_content.loc[ind, 'id'] for ind in pred.get()])
        predictions.append(p)
    tmp_topics['predictions'] = predictions
    # Release memory
    del topics_preds_gpu, content_preds_gpu, neighbors_model, predictions, indices, model
    gc.collect()
    torch.cuda.empty_cache()
    return tmp_topics, tmp_content

# =========================================================================================
# Process test
# =========================================================================================
def preprocess_test(tmp_test):
    tmp_test['title1'].fillna("Title does not exist", inplace = True)
    tmp_test['title2'].fillna("Title does not exist", inplace = True)
    # Create feature column
    tmp_test['text'] = tmp_test['title1'] + '[SEP]' + tmp_test['title2']
    # Drop titles
    tmp_test.drop(['title1', 'title2'], axis = 1, inplace = True)
    # Sort so inference is faster
    tmp_test['length'] = tmp_test['text'].apply(lambda x: len(x))
    tmp_test.sort_values('length', inplace = True)
    tmp_test.drop(['length'], axis = 1, inplace = True)
    tmp_test.reset_index(drop = True, inplace = True)
    gc.collect()
    torch.cuda.empty_cache()
    return tmp_test

# =========================================================================================
# Model
# =========================================================================================
class custom_model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.config = AutoConfig.from_pretrained(cfg.sup_model + '/config', output_hidden_states = True)
        self.config.hidden_dropout = 0.0
        self.config.hidden_dropout_prob = 0.0
        self.config.attention_dropout = 0.0
        self.config.attention_probs_dropout_prob = 0.0
        self.model = AutoModel.from_pretrained(cfg.sup_model + '/model', config = self.config)
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        feature = self.pool(last_hidden_state, inputs['attention_mask'])
        return feature
    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output
    
# =========================================================================================
# Inference function loop
# =========================================================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total = len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().squeeze().to('cpu').numpy().reshape(-1))
    predictions = np.concatenate(preds)
    return predictions

# =========================================================================================
# Inference
# =========================================================================================
def inference(tmp_test, cfg, _idx):
    # Create dataset and loader
    test_dataset = sup_dataset(tmp_test, cfg)
    test_loader = DataLoader(
        test_dataset, 
        batch_size = 32, 
        shuffle = False, 
        collate_fn = DataCollatorWithPadding(tokenizer = cfg.sup_tokenizer, padding = 'longest'),
        num_workers = 4, 
        pin_memory = True, 
        drop_last = False
    )
    # Get model
    model = custom_model(cfg)
    # Load weights
    state = torch.load("/kaggle/input/20230123/xlm-roberta-base_fold0_42.pth", map_location = torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    # Release memory
    del test_dataset, test_loader, model, state
    gc.collect()
    torch.cuda.empty_cache()
    # Use threshold
    tmp_test['predictions'] = np.where(prediction > 0.0006, 1, 0)
    tmp_test1 = tmp_test[tmp_test['predictions'] == 1]
    tmp_test1 = tmp_test1.groupby(['topics_ids'])['content_ids'].unique().reset_index()
    tmp_test1['content_ids'] = tmp_test1['content_ids'].apply(lambda x: ' '.join(x))
    tmp_test1.columns = ['topic_id', 'content_ids']
    tmp_test0 = pd.Series(tmp_test['topics_ids'].unique())
    tmp_test0 = tmp_test0[~tmp_test0.isin(tmp_test1['topic_id'])]
    tmp_test0 = pd.DataFrame({'topic_id': tmp_test0.values, 'content_ids': ""})
    tmp_test_r = pd.concat([tmp_test1, tmp_test0], axis = 0, ignore_index = True)
    tmp_test_r.to_csv(f'submission_{_idx + 1}.csv', index = False)
    print(tmp_test_r.head())
    
    del tmp_test, tmp_test1, tmp_test0, tmp_test_r
    gc.collect()
    torch.cuda.empty_cache()

# Retrievers Ensemble

In [3]:
for _idx, CFG in enumerate(CFG_list):
    # Read data
    tmp_topics, tmp_content = read_data(CFG)
    # Run nearest neighbors
    tmp_topics, tmp_content = get_neighbors(tmp_topics, tmp_content, CFG)
    gc.collect()
    torch.cuda.empty_cache()
    # Set id as index for content
    tmp_content.set_index('id', inplace = True)
    # Build training set
    tmp_test = build_inference_set(tmp_topics, tmp_content, CFG)
    # Process test set
    tmp_test = preprocess_test(tmp_test)
    # Inference
    inference(tmp_test, CFG, _idx)
    del tmp_topics, tmp_content, tmp_test
    gc.collect()
    torch.cuda.empty_cache()
    
df_test = pd.concat([pd.read_csv(f'submission_{_idx + 1}.csv') for _idx in range(len(CFG_list))])
df_test.fillna("", inplace = True)
df_test['content_ids'] = df_test['content_ids'].apply(lambda c: c.split(' '))
df_test = df_test.explode('content_ids').groupby(['topic_id'])['content_ids'].unique().reset_index()
df_test['content_ids'] = df_test['content_ids'].apply(lambda c: ' '.join(c))

df_test.to_csv('submission.csv', index = False)
df_test.head()

 
--------------------------------------------------
topics.shape: (5, 2)
content.shape: (154047, 2)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4814 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

 
Training KNN model...


...

 
Training KNN model...


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

         topic_id                                        content_ids
0  t_00004da3a1b2                      c_1108dd0c7a5d c_5bc0e1e2cba0
1  t_00068291e9a4  c_ebb7fdf10a7e c_639ea2ef9c95 c_14bf71640ecd c...
2  t_0006d41a73a8  c_b972646631cb c_0c6473c3480d c_d7a0d7eaf799 c...
3  t_4054df11a74e                      c_f2d184a98231 c_3695c5dc1df6
4  t_00069b63a70a                                                   


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_5bc0e1e2cba0
1,t_00068291e9a4,c_ebb7fdf10a7e c_14bf71640ecd c_ac1672cdcd2c c...
2,t_00069b63a70a,
3,t_0006d41a73a8,c_b972646631cb c_0c6473c3480d c_d7a0d7eaf799 c...
4,t_4054df11a74e,c_f2d184a98231 c_3695c5dc1df6
